<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

# Emotion Classifier

In [ ]:
%%capture
!pip install pytorch-lightning==1.3.8 torchviz wandb
!git clone https://github.com/wandb/lit_utils
!cd "/content/lit_utils" && git pull

import pytorch_lightning as pl
import torch
import wandb

import lit_utils as lu

lu.utils.filter_warnings()

## Defining the `Model`

In [ ]:
class LitEmotionClassifier(lu.nn.modules.LoggedImageClassifierModule):

  def __init__(self, config, max_images_to_display=32):
    super().__init__()

    self.labels = ["Angry", "Disgusted", "Afraid", "Happy",
                "Sad", "Surprised", "Neutral"]

    # first, convolutional component
    self.conv_layers = torch.nn.Sequential(*[                    
    ])

    # need a fixed-size input for fully-connected component,
    #  so apply a "re-sizing" layer, to size set in config
    self.resize_layer = torch.nn.AdaptiveAvgPool2d(
      (config["final_height"], config["final_width"]))
    
    # now, we can apply our fully-connected component
    final_size = config["final_height"] * config["final_width"] * config["conv.channels"][-1]
    self.fc_layers = torch.nn.Sequential(*[
        lu.nn.fc.FullyConnected(
            in_features=final_size,
            out_features=config["output.size"]),                                                           
    ])

    self.optimizer = config["optimizer"]
    self.optimizer_params = config["optimizer.params"]
    self.loss = config["loss_fn"]

  def forward(self, x):
    # first apply convolutional layers
    for layer in self.conv_layers: 
      x = layer(x)

    # then convert to a fixed-size vector
    x = self.resize_layer(x)
    x = torch.flatten(x, start_dim=1)

    # then apply the fully-connected layers
    for layer in self.fc_layers: # snap together the LEGOs
      x = layer(x)
    
    return x

## Building the `Model` and Loading the Data

In [ ]:
config = {
  "batch_size": 256,
  "max_epochs": 10,
  "conv.channels": [1],
  "final_height": 24,
  "final_width": 24,
  "fc.size": [],
  "activation": torch.nn.ReLU(),
  "loss_fn": torch.nn.CrossEntropyLoss(),  # cross-entropy loss
  "optimizer": torch.optim.Adam,
  "optimizer.params": {"lr": 0.001},
}


dmodule = lu.datamodules.FERDataModule(batch_size=config["batch_size"])
config["output.size"] = len(dmodule.classes)
lec = LitEmotionClassifier(config)
dmodule.setup()
dmodule.prepare_data()

### Debugging Code

In [ ]:
# for debugging purposes (checking shapes, etc.), make these available
dloader = dmodule.train_dataloader()  # set up the Loader

example_batch = next(iter(dloader))  # grab a batch from the Loader
example_x, example_y = example_batch[0].to("cuda"), example_batch[1].to("cuda")

print(f"Input Shape: {example_x.shape}")
print(f"Target Shape: {example_y.shape}")

lec.to("cuda")
outputs = lec.forward(example_x)
print(f"Output Shape: {outputs.shape}")
print(f"Loss: {lec.loss(outputs, example_y)}")

### Running `.fit`

In [ ]:
with wandb.init(project="lit-fer", entity="wandb", config=config):
  # 🪵 configure logging
  cbs=[lu.callbacks.WandbCallback(),  # callbacks add extra features, like better logging
       lu.callbacks.FilterLogCallback(image_size=(config["final_height"], config["final_width"]), log_input=True),  # this one logs the weights as images
       lu.callbacks.ImagePredLogCallback(labels=dmodule.classes, on_train=True)  # and this one logs the inputs and outputs
       ]
  wandblogger = pl.loggers.WandbLogger(save_code=True)

  # 👟 configure Trainer 
  trainer = pl.Trainer(gpus=1,  # use the GPU for .forward
                       logger=wandblogger,  # log to Weights & Biases
                       callbacks=cbs,  # use callbacks to log lots of run data
                       max_epochs=config["max_epochs"], log_every_n_steps=1,
                       progress_bar_refresh_rate=50)

  # 🏃‍♀️ run the Trainer on the model
  trainer.fit(lec, datamodule=dmodule)